# Visualización de Incendios Forestales

In [4]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Dataframe
url = 'https://raw.githubusercontent.com/AndrsGzRo/incendios-forestales-mx/refs/heads/main/data-2025-11-05.csv'
df = pd.read_csv(url)

# Primeros cincos renglones
df.head()

,anio,Clave_del_incendio,latitud,longitud,Clave_Municipio,Estado,Municipio,CVE_ENT,CVE_MUN,CVEGEO,...,Entidad,fn_Clave_del_incendio,fn_Predio,fn_Causa,fn_Causa_especifica,fn_Duracion_dias,fn_Tipo_de_incendio,fn_Tipo_Vegetacion,fn_Tipo_impacto,fn_Tamano
0,2015,15-01-0001,21.995503,-102.750478,3.0,Aguascalientes,Calvillo,1,3.0,301,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,15-01-0002,21.773878,-102.300881,1.0,Aguascalientes,Aguascalientes,1,1.0,101,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,15-01-0003,21.785075,-102.319444,1.0,Aguascalientes,Aguascalientes,1,1.0,101,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,15-01-0004,22.224067,-102.336411,7.0,Aguascalientes,Rincón de Romos,1,7.0,701,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,15-01-0005,22.041083,-101.973917,2.0,Aguascalientes,Asientos,1,2.0,201,...,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.1 Información de dataframe

In [11]:
def info_df(df):
    print('---------Información general del Dataframe---------')
    print(df.info())
    print('---------Estadísticas del df---------')
    print(df.describe())
    print('---------Valores nulos---------')
    print(df.isnull().sum().sort_values(ascending=False))
    print('---------Valores únicos---------')
    print(df.nunique())

info_df(df)

---------Información general del Dataframe---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71089 entries, 0 to 71088
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   anio                   71089 non-null  int64  
 1   Clave_del_incendio     71089 non-null  object 
 2   latitud                71089 non-null  float64
 3   longitud               71089 non-null  float64
 4   Clave_Municipio        71075 non-null  float64
 5   Estado                 71089 non-null  object 
 6   Municipio              71089 non-null  object 
 7   CVE_ENT                71089 non-null  int64  
 8   CVE_MUN                71089 non-null  float64
 9   CVEGEO                 71089 non-null  int64  
 10  Region                 71089 non-null  object 
 11  Predio                 71087 non-null  object 
 12  Causa                  71089 non-null  object 
 13  Causa_especifica       71089 non-null  object 
 14  Fe

## 1.2 Limpieza

Se eliminarán las variables que empiezan con ``` fn_``` ya que suelen ser valores nulos.




In [21]:
# Eliminando columnas fn
df_limpio = df.loc[:,~df.columns.str.startswith('fn_')]
info_df(df_limpio)

---------Información general del Dataframe---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71089 entries, 0 to 71088
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   anio                71089 non-null  int64  
 1   Clave_del_incendio  71089 non-null  object 
 2   latitud             71089 non-null  float64
 3   longitud            71089 non-null  float64
 4   Clave_Municipio     71075 non-null  float64
 5   Estado              71089 non-null  object 
 6   Municipio           71089 non-null  object 
 7   CVE_ENT             71089 non-null  int64  
 8   CVE_MUN             71089 non-null  float64
 9   CVEGEO              71089 non-null  int64  
 10  Region              71089 non-null  object 
 11  Predio              71087 non-null  object 
 12  Causa               71089 non-null  object 
 13  Causa_especifica    71089 non-null  object 
 14  Fecha_Inicio        71086 non-null  float64
 15  F

## 1.3 Transformación de Variables

#### 1.3.1 Variables Temporales
Las variables temporales son:
- ```anio```
- ```Fecha_Inicio```
- ```Fecha_Termino```
- ```Duracion_dias```

Las variables anteriores tienen un formato correcto, excepto ```Duracion_dias```, la cual será transformada a una ordinal.


In [26]:
mapa_duraciones = {
    '1 Día': 1,
    '2 a 3 Días': 2.5,
    '4 a 7 Días':5.5,
    'Mayor de 7 Días':10
}
df_limpio['Duracion_dias_num'] = df_limpio['Duracion_dias'].map(mapa_duraciones)